<a href="https://colab.research.google.com/github/li4520/youtube-summarization/blob/main/YT_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers youtube-transcript-api sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.7 MB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi as YTapi
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
youtube_link = "https://www.youtube.com/watch?v=DNjlqqkvm_s"

In [ ]:
def videoID(link):
    video_id = link.split("=")[1]
    return video_id

In [ ]:
id = videoID(youtube_link)

In [ ]:
id

'DNjlqqkvm_s'

In [ ]:
def GetTranscript(video_id):
    try:
        transcript = YTapi.get_transcript(video_id)
        FinalTranscript = ' '.join([i['text'] for i in transcript])
    except Exception as e:
        print(e)
        
    return FinalTranscript

In [ ]:
transcript_en = GetTranscript(id)

In [ ]:
transcript_en

"Netflix password sharing.\nWhy are you freaking out about it? Okay, let's get to the facts, because\nlast year Netflix told us more than 100 million households currently use\nmemberships and Netflix don't pay for it.\nAnd they wanted to crack down on this post about sharing.\nProbably coming into effect at the end of the first quarter.\nThat means by March. Now, why are we all freaking out right\nhere right now? Well, they've already been trialing ways\nto curtail possible sharing over in Latin America in three key countries, in\nfact, Peru, Chile, Costa Rica. And they put out a helpful article to\ntell users in those countries about some of the changes.\nThey published it in every single region.\nSo that's why we're all worrying about it.\nSo we're always seeing being trialled in those three countries.\nWell, the users are being asked to set a primary location for where they use\nNetflix on the devices. And then Netflix will track a mixture of\nIP addresses or device I.D. account act

In [ ]:
transcript_list  = YTapi.list_transcripts(id)

for transcript in transcript_list:
    ln = transcript.language
    check = transcript.is_translatable    
    print(ln, check)

English True
English (auto-generated) True


In [ ]:
for transcript in transcript_list:
    available_ln = transcript.translation_languages
print(available_ln)

[{'language': 'Afrikaans', 'language_code': 'af'}, {'language': 'Akan', 'language_code': 'ak'}, {'language': 'Albanian', 'language_code': 'sq'}, {'language': 'Amharic', 'language_code': 'am'}, {'language': 'Arabic', 'language_code': 'ar'}, {'language': 'Armenian', 'language_code': 'hy'}, {'language': 'Assamese', 'language_code': 'as'}, {'language': 'Aymara', 'language_code': 'ay'}, {'language': 'Azerbaijani', 'language_code': 'az'}, {'language': 'Bangla', 'language_code': 'bn'}, {'language': 'Basque', 'language_code': 'eu'}, {'language': 'Belarusian', 'language_code': 'be'}, {'language': 'Bhojpuri', 'language_code': 'bho'}, {'language': 'Bosnian', 'language_code': 'bs'}, {'language': 'Bulgarian', 'language_code': 'bg'}, {'language': 'Burmese', 'language_code': 'my'}, {'language': 'Catalan', 'language_code': 'ca'}, {'language': 'Cebuano', 'language_code': 'ceb'}, {'language': 'Chinese (Simplified)', 'language_code': 'zh-Hans'}, {'language': 'Chinese (Traditional)', 'language_code': 'zh-

In [ ]:
def summarize(tokenizer, model, text):
    
    """
    Function to summarize the text(transcript)
    """
    inputs = tokenizer(text, 
                    max_length=1024, 
                    truncation=True,
                    return_tensors="pt")
  
    summary_ids = model.generate(inputs["input_ids"])
    summary = tokenizer.batch_decode(summary_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)
    return summary

In [ ]:
summary1 = "google/pegasus-large"
summary2 = "sshleifer/distilbart-cnn-12-6"

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained(summary1)
model1 = AutoModelForSeq2SeqLM.from_pretrained(summary1)

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained(summary2)
model2 = AutoModelForSeq2SeqLM.from_pretrained(summary2)

In [ ]:
pegasus = summarize(tokenizer1, model1, transcript_en)
pegasus[0]


"Okay, let's get to the facts, because last year Netflix told us more than 100 million households currently use memberships and Netflix don't pay for it. Well, they've already been trialing ways to curtail possible sharing over in Latin America in three key countries, in fact, Peru, Chile, Costa Rica. Well, the users are being asked to set a primary location for where they use Netflix on the devices. Say they wanted to talk about Netflix, spatial audio, a better amount of more devices if you have the premium Netflix subscription and instead it got drowned out by this. And I think now we're going to have to hear from Netflix about how this might be enacted, the trials that were going on and what it's actually going to mean for you and me eventually in the U.S.."

In [ ]:
bart = summarize(tokenizer2, model2, transcript_en)
bart[0]


' Netflix has been trialing ways to curtail possible sharing over in Latin America in three key countries, Peru, Chile, Costa Rica . Users are being asked to set a primary location for where they use Netflix on the devices . Netflix will track a mixture of .IP addresses or device I.D. account activity to see if anyone outside the household is using that password .'